In [6]:
from src import cropy as cpy
import json
import geopandas as gpd
import numpy as np
import rioxarray
import xarray as xr
#https://nbviewer.jupyter.org/github/pangeo-data/cog-best-practices/blob/main/0-single-cog.ipynb
import os
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='8000000000'  #800MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='8000000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='8000000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE8

In [7]:
target_aoi={
"type": "FeatureCollection",
"name": "Harran_AOI",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "fid": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 38.680367610550277, 36.71081864055018 ], [ 38.688923838082594, 37.274751818816462 ], [ 39.282414893278698, 37.277085335416182 ], [ 39.29019328194444, 36.690594830019251 ], [ 38.680367610550277, 36.71081864055018 ] ] ] } }
]
}


In [8]:
datajson=json.dumps(target_aoi)
area=gpd.read_file(datajson)


In [9]:
cpy.VectorProcessing.show_vector(target_area=area)

In [10]:
m,intersec_df=cpy.VectorProcessing.show_intersection(target_area=area,base_vector_path='sentinel_tiles/sentinel_tr_tiles.shp')

In [11]:
m

In [12]:
boundry=list(area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2018-08-01/2018-12-01'
band_list=['B02','B08']
cloud_percentage=30

In [13]:
stac_query=cpy.Stac(target_aoi=bbox,date=dates,max_cloud=cloud_percentage)

In [14]:
sentinel_items=stac_query.find_sentinel_item()

In [15]:
sentinel_items

[S2A_37SDA_20181030_0_L2A,
 S2B_37SDB_20180925_0_L2A,
 S2A_37SEA_20181030_0_L2A,
 S2A_37SEB_20181030_0_L2A]

In [16]:
cpy.Stac.show_result_map(items_list=sentinel_items,
                         overview=True,
                         target_area=target_aoi)

In [ ]:
cpy.Stac.show_result_list(sentinel_items)[0:2]

In [ ]:
cpy.Stac.show_result_df(sentinel_items_list=sentinel_items)

In [18]:
band_list=['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A',
           'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'info', 'metadata',
           'visual', 'overview', 'thumbnail']

In [ ]:
%%time
#download all sentinel_items with target bands
cpy.Stac.download_image(item_list=sentinel_items,
                        band_list=band_list[0:4],                                           
                        download_path='./target_path2'
                                              )

In [ ]:
%%time
# get data as xarray or open download_status and save the data to download_path
subset_image=stac_query.download_subset_image(item_list=sentinel_items,
                                              band_list=['B02', 'B08','B03','B04'],
                                              aoi=target_aoi,
                                              download_status=True,
                                              download_path='./target_path2_subset'
                                              )

In [ ]:
%%time
#download all sentinel_items with target bands
cpy.Stac.download_image(item_list=sentinel_items,
                        band_list=band_list[0:4],
                        download_path='./target_path2'
                        )

In [ ]:
%%time
# get data as xarray and open download_status and save the data to download_path
subset_image=stac_query.download_subset_image(item_list=sentinel_items,
                                              band_list=['B02', 'B08','B03','B04'],
                                              aoi=target_aoi,
                                              download_status=True,
                                              download_path='./target_path3'
                                              )

### FIND TIME SERIES DATA

In [13]:
from src import cropy as cpy
import json
import geopandas as gpd
import numpy as np
target_aoi={
"type": "FeatureCollection",
"name": "Harran_AOI",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "fid": 1 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 38.680367610550277, 36.71081864055018 ], [ 38.688923838082594, 37.274751818816462 ], [ 39.282414893278698, 37.277085335416182 ], [ 39.29019328194444, 36.690594830019251 ], [ 38.680367610550277, 36.71081864055018 ] ] ] } }
]
}

datajson=json.dumps(target_aoi)
area=gpd.read_file(datajson)

#cpy.VectorProcessing.show_vector(target_area=area)
m,intersec_df=cpy.VectorProcessing.show_intersection(target_area=area,base_vector_path='sentinel_tiles/sentinel_tr_tiles.shp')
#m
boundry=list(area.geometry.bounds.values[0]) #boundry from your AOI
bbox=[boundry[0],boundry[1],boundry[2],boundry[3]] #(min lon, min lat, max lon, max lat)
dates = '2018-08-01/2018-12-01'
band_list=['B02','B08']
cloud_percentage=30


In [107]:
stac_query=cpy.Stac(target_aoi=bbox,date=dates,max_cloud=cloud_percentage)


In [108]:
t_list=stac_query.create_tiles_list()
t_list

['37SDA', '37SDB', '37SEA', '37SEB']

In [109]:
stac_query.tiles_list=['37SDA', '37SDB', '37SEA']
stac_query.find_time_series()


In [110]:
#cpy.Stac.show_result_list(stac_query.stac_items)
res_df=cpy.Stac.show_result_df(items_list=stac_query.stac_items)

In [105]:
res_df.head(2)

,datetime,platform,constellation,instruments,gsd,view:off_nadir,proj:epsg,sentinel:utm_zone,sentinel:latitude_band,sentinel:grid_square,sentinel:sequence,sentinel:product_id,sentinel:data_coverage,eo:cloud_cover,sentinel:valid_cloud_cover,created,updated,geometry,data_coverage
0,2018-11-09 08:19:51,sentinel-2a,sentinel-2,msi,10,0,32637,37,S,DA,0,S2A_MSIL2A_20181109T081141_N0210_R078_T37SDA_2...,100,14.75,True,2020-08-31T03:04:52.022Z,2020-08-31T03:04:52.022Z,"POLYGON ((37.88933 36.05159, 37.87508 37.04124...",NaN
1,2018-10-30 08:12:29,sentinel-2a,sentinel-2,msi,10,0,32637,37,S,DA,0,S2A_MSIL2A_20181030T081041_N0209_R078_T37SDA_2...,100,0.00,True,2020-09-26T02:48:18.340Z,2020-09-26T02:48:18.340Z,"POLYGON ((37.88933 36.05159, 37.87508 37.04124...",100.0


In [65]:
cpy.Stac.show_result_map(items_list=stac_query.stac_items,
                         overview=True,
                         target_area=target_aoi)

In [23]:
tar_items_list=list(res_df['sentinel:product_id'])[0:6]

In [45]:
items_list=['S2B_MSIL2A_20180915T080559_N0208_R078_T37SDA_20180915T123831','S2A_MSIL2A_20180910T080601_N0208_R078_T37SDA_20180910T130532']

In [24]:
cpy.Stac.download_image(stac_result=stac_query.stac_result,
                        item_id_list=tar_items_list,
                        band_list=['B04','B08'],
                        download_path='time_series_path')

'time_series_path'

In [48]:
%%time
subset_2bands=cpy.Stac.download_subset_image(stac_result=stac_query.stac_result,
                                             item_id_list=tar_items_list,
                                             aoi=target_aoi,
                                             band_list=['B04','B08'],)

CPU times: user 12.4 s, sys: 955 ms, total: 13.4 s
Wall time: 23.2 s


### Calculate Image Statistic
Below function, calculate the statistic of each band in subset image list. <br> 
It'll be more real example when you use this method with time series data example <br>
On the same Sentinel-2 tile, you can get stat (mean,median,min,max) value at your target time range

In [51]:
%%time
st=cpy.calc_img_stat(stac_result=subset_2bands,statistic_method='median')

CPU times: user 25.6 s, sys: 10.8 s, total: 36.4 s
Wall time: 14min 48s


In [55]:
st

{'B04': <xarray.DataArray (y: 3878, x: 3831)>
 array([[   0,    0,    0, ..., 1523, 2068, 2636],
        [   0,    0,    0, ..., 1487, 1959, 2733],
        [   0,    0,    0, ..., 1589, 1986, 2756],
        ...,
        [   0,    0,    0, ...,  723,  716,  726],
        [   0,    0,    0, ...,  738,  727,  721],
        [   0,    0,    0, ...,  730,  730,  741]], dtype=uint16)
 Coordinates:
   * y            (y) float64 4.1e+06 4.1e+06 4.1e+06 ... 4.061e+06 4.061e+06
   * x            (x) float64 4.715e+05 4.715e+05 ... 5.097e+05 5.098e+05
     spatial_ref  int64 0
 Attributes:
     _FillValue:  0,
 'B08': <xarray.DataArray (y: 3878, x: 3831)>
 array([[   0,    0,    0, ..., 2521, 3060, 3744],
        [   0,    0,    0, ..., 2483, 3074, 3690],
        [   0,    0,    0, ..., 2525, 3003, 3525],
        ...,
        [   0,    0,    0, ..., 2460, 2401, 2369],
        [   0,    0,    0, ..., 2427, 2363, 2414],
        [   0,    0,    0, ..., 2449, 2459, 2430]], dtype=uint16)
 Coordinates:
